<a href="https://colab.research.google.com/github/coltongerth/rpms-degradation/blob/main/rpms_degradation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [46]:
%%capture
!pip install pandas
!pip install joblib
!pip install numpy
!pip install scipy
!pip install statsmodels

In [21]:
# Clear any existing logging configurations
for handler in logging.root.handlers[:]:
    logging.root.removeHandler(handler)

# Set up basic logging to display in the notebook
logging.basicConfig(
    level=logging.INFO,  # Set to DEBUG for more detailed output
    format='%(asctime)s - %(levelname)s - %(message)s',  # Write logs to a file
    handlers=[
        logging.FileHandler("parallel_logs.log"),
        logging.StreamHandler()  # Ensures logs show up in the notebook
    ]
)

In [63]:
import os
import time
import multiprocessing
import logging
import pandas as pd
import numpy as np
from google.colab import drive
from datetime import datetime
from statsmodels.regression.mixed_linear_model import MixedLM
from scipy.stats import t, linregress
from joblib import Parallel, delayed
from scipy import stats
# from statsmodels.stats.correlation_tools import corARMA
from statsmodels.tsa.arima_model import ARMA
from statsmodels.regression.linear_model import GLS

drive.mount('/content/gdrive')





class FDR:
    def __init__(self, pvals, qlevel=0.05, method="original", adjustment_method=None, adjustment_args=None):
        self.pvals = np.array(pvals)
        self.qlevel = qlevel
        self.method = method
        self.adjustment_method = adjustment_method
        self.adjustment_args = adjustment_args or {}

    def calculate(self):
        n = len(self.pvals)
        a = 0  # Initialize proportion of alternative hypotheses

        if self.adjustment_method is not None:
            if self.adjustment_method == "two-stage":
                self.qlevel = self.qlevel / (1 + self.qlevel)
                self.adjustment_args['qlevel'] = self.qlevel
                self.adjustment_args['fdr_method'] = self.method
                print(f"Adjusting cutoff using two-stage method with method {self.method} and qlevel {round(self.qlevel, 4)}")

            if self.adjustment_method == "mean" and not self.adjustment_args:
                self.adjustment_args = {'edf_lower': 0.8, 'num_steps': 20}
                print(f"Adjusting cutoff using mean method with edf_lower=0.8 and num_steps=20")

            a = self._prop_alt(self.pvals, self.adjustment_method, self.adjustment_args)

        if a == 1:  # All hypotheses are estimated to be alternatives
            return np.arange(1, n + 1)
        else:
            self.qlevel = self.qlevel / (1 - a)

        return self._fdr_master(self.pvals, self.qlevel, self.method)

    def _fdr_master(self, pvals, qlevel, method):
        n = len(pvals)
        if method == "general":
            qlevel = qlevel / np.sum(1 / np.arange(1, n + 1))  # More conservative method
        elif method != "original":
            raise ValueError(f"No method of type: {method}")

        return self._fdr_basic(pvals, qlevel)

    def _fdr_basic(self, pvals, qlevel):
        n = len(pvals)
        sorted_pvals = np.sort(pvals)
        sort_index = np.argsort(pvals)
        indices = np.arange(1, n + 1) * (sorted_pvals <= qlevel * np.arange(1, n + 1) / n)
        num_reject = np.max(indices)

        if num_reject > 0:
            indices = np.arange(1, num_reject + 1)
            return np.sort(sort_index[:num_reject])
        else:
            return None

    def _prop_alt(self, pvals, adjustment_method="mean", adjustment_args=None):
        adjustment_args = adjustment_args or {'edf_lower': 0.8, 'num_steps': 20}
        n = len(pvals)

        if adjustment_method == "two-stage":
            if 'qlevel' not in adjustment_args or 'fdr_method' not in adjustment_args:
                raise ValueError("adjustment_args must include qlevel and fdr_method for two-stage method.")
            return len(self._fdr_master(pvals, adjustment_args['qlevel'], adjustment_args['fdr_method'])) / n

        if adjustment_method == "storey":
            if 'edf_quantile' not in adjustment_args:
                raise ValueError("adjustment_args must include edf_quantile for Storey's method.")
            return self._storey(adjustment_args['edf_quantile'], pvals)

        if adjustment_method == "mean":
            edf_lower = adjustment_args.get('edf_lower', 0.8)
            num_steps = adjustment_args.get('num_steps', 20)

            if edf_lower >= 1 or edf_lower <= 0:
                raise ValueError("edf_lower must be between 0 and 1")
            if num_steps < 1 or not isinstance(num_steps, int):
                raise ValueError("num_steps must be a positive integer")

            step_size = (1 - edf_lower) / num_steps
            edf_quantiles = np.linspace(edf_lower, 1 - step_size, num_steps)
            a_vec = np.array([self._storey(q, pvals) for q in edf_quantiles])
            return np.mean(a_vec)

    def _storey(self, edf_quantile, pvals):
        if edf_quantile >= 1 or edf_quantile <= 0:
            raise ValueError("edf_quantile should be between 0 and 1")

        a = (np.mean(pvals <= edf_quantile) - edf_quantile) / (1 - edf_quantile)
        return max(a, 0)




# Example of manually calculating AR(1) covariance matrix for GLS
def ar1_cov_matrix(n, rho):
    cov_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            cov_matrix[i, j] = rho ** abs(i - j)
    return cov_matrix

# Read in the data
rpms_data = pd.read_csv("rpms_stack_60m_cut_filtered_mean_filled_zones_filtered_100.csv")
zone_data = pd.read_csv("BpsMskR2Fin_60m_cut.csv")

# Merge the RPMs data with zone data
rpms_data = pd.merge(zone_data, rpms_data, on=["X", "Y"])
print(rpms_data)

# Replace zero values with the row mean
def replace_zeros_with_row_mean(row):
    # Calculate the row mean, ignoring zeros
    row_mean = row[3:].replace(0, np.nan).mean()
    # Replace zeros with the row mean
    return row.apply(lambda x: row_mean if x == 0 else x)

rpms_data.iloc[:, 3:] = rpms_data.iloc[:, 3:].apply(replace_zeros_with_row_mean, axis=1)

nyears = 39
year = np.arange(1, nyears + 1)

# Detect and register parallel backend
n_cores = multiprocessing.cpu_count()
print(f"Using {n_cores} cores")

# Start time for performance monitoring

start_time = time.time()

# Perform analysis for each zone separately
zones = rpms_data['Zone'].unique()
# print(zones)
logging.info(f"Zones to be processed: {zones}")
def process_zone(zone, rpms_data, nyears):
    # Subset data for current zone
    subset_zone_data = rpms_data[rpms_data['Zone'] == zone]

    zone_results = []
    print(subset_zone_data)
    for i in range(len(subset_zone_data)):
        # Exclude the current row
        # zone_data_excl = subset_zone_data.drop(subset_zone_data.index[i])

        # if zone_data_excl.isna().all().all():
        #     continue

        # Create p1 data
        p1 = pd.melt(subset_zone_data, id_vars=['X', 'Y', 'Zone'], var_name='description', value_name='y')
        p1['x'] = np.tile(np.arange(1, nyears + 1), len(subset_zone_data))

        # Mean and trend models
        p1_mean = np.mean(p1['y'])
        p1_trend = np.polyfit(p1['x'], p1['y'], 1)

        # Extract current row y values
        y = subset_zone_data.iloc[i, 3:nyears+3].values

        # Ensure that y has the same length as the time variable
        # if len(y) != nyears:
        #     logging.warning(f"Skipping row {i} due to length mismatch between y and nyears")
        #     continue
        print("y",y)
        if np.all(y == 0):
            continue

        rho = 0.5  # Autoregressive parameter
        cov_matrix = ar1_cov_matrix(len(y), rho)

        # Field mean and trend models
        # try:
        nyears = len(y)
        print(len(y))
        mean_fit = GLS(y, np.ones_like(y), sigma=cov_matrix).fit()
        trend_fit = GLS(y, np.column_stack((np.ones_like(y), np.arange(1, nyears + 1))), sigma=cov_matrix).fit()
        # except ValueError as e:
        #     logging.error(f"Error in GLS fitting for row {i}: {e}")
        #     continue

        degf_mean = mean_fit.df_resid
        degf_trend = trend_fit.df_resid

        # Create results dataframe for the current row
        z1 = pd.DataFrame({
            'X': [subset_zone_data.iloc[i, 0]],
            'Y': [subset_zone_data.iloc[i, 1]],
            'Zone': [subset_zone_data.iloc[i, 2]],
            'mean_field': [np.mean(y)],
            'mean_population': [p1_mean],
            'mean_difference': [np.mean(y) - p1_mean],
            'se_mean': [np.sqrt(mean_fit.bse[0])],
            't_stat_mean': [(np.mean(y) - p1_mean) / np.sqrt(mean_fit.bse[0])],
            'p_value_mean': [2 * (1 - stats.t.cdf(abs((np.mean(y) - p1_mean) / np.sqrt(mean_fit.bse[0])), degf_mean))],
            'adj_p_value_mean': [np.nan],
            'slope_field': [trend_fit.params[1]],
            'slope_population': [p1_trend[0]],
            'slope_difference': [trend_fit.params[1] - p1_trend[0]],
            'se_slope': [np.sqrt(trend_fit.bse[1])],
            't_stat_slope': [(trend_fit.params[1] - p1_trend[0]) / np.sqrt(trend_fit.bse[1])],
            'p_value_slope': [2 * (1 - stats.t.cdf(abs((trend_fit.params[1] - p1_trend[0]) / np.sqrt(trend_fit.bse[1])), degf_trend))],
            'adj_p_value_slope': [np.nan]
        })
        print(z1)
        zone_results.append(z1)

    return pd.concat(zone_results, ignore_index=True)

    # except Exception as e:
    #     logging.error(f"Error processing zone {zone}: {str(e)}")
    #     return pd.DataFrame()  # Return empty DataFrame in case of an error

# Sequential execution for each zone
results_list = []
for zone in zones:
    logging.info(f"Processing zone: {zone}")
    result = process_zone(zone,rpms_data,nyears)
    results_list.append(result)

print("results_list", results_list)
# Combine all results
final_results = pd.concat(results_list)
print("final_results", final_results)

# Apply False Discovery Rate (FDR) adjustment (you would need an FDR function here)
def apply_fdr_by_zone(subset):
    # Check if the subset is empty or None
    if subset is None or subset.shape[0] == 0:
        return None  # Return None if the subset is empty

    # Extract p-values for means (9th column)
    pvals_mean = subset.iloc[:, 8].values  # 9th column (index 8) in zero-indexed Python
    fdr_instance_mean = FDR(pvals=pvals_mean, qlevel=0.2, method="general", adjustment_method="mean")
    significant_mean_indices = fdr_instance_mean.calculate()

    # Create a boolean array where True means the test was significant
    significant_mean = np.ones(len(pvals_mean), dtype=bool)
    if significant_mean_indices is not None:
        print(significant_mean)
        significant_mean[significant_mean_indices] = False

    # Extract p-values for slopes (16th column)
    pvals_slope = subset.iloc[:, 15].values  # 16th column (index 15) in zero-indexed Python
    fdr_instance_slope = FDR(pvals=pvals_slope, qlevel=0.2, method="general", adjustment_method="mean")
    significant_slope_indices = fdr_instance_slope.calculate()

    # Create a boolean array where True means the test was significant
    significant_slope = np.ones(len(pvals_slope), dtype=bool)
    if significant_slope_indices is not None:
        significant_slope[significant_slope_indices] = False

    # Update the subset DataFrame with the adjusted significance values
    subset.iloc[:, 9] = significant_mean  # Column 10 in R (9 in Python)
    subset.iloc[:, 16] = significant_slope  # Column 17 in R (16 in Python)

    return subset


def apply_fdr_to_results(results_list):
    # Apply FDR adjustment for each zone
    adjusted_results_list = []
    for subset in results_list:
        adjusted_subset = apply_fdr_by_zone(subset)
        if adjusted_subset is not None:
            adjusted_results_list.append(adjusted_subset)

    # Combine all adjusted subsets back into one DataFrame
    if adjusted_results_list:
        return pd.concat(adjusted_results_list, ignore_index=True)
    else:
        return pd.DataFrame()  # Return empty DataFrame if no significant results

# After calculating the results for all zones, apply FDR correction
final_results_with_fdr = apply_fdr_to_results(results_list)
print(final_results_with_fdr)
# Drop extra zone column added to the front from the function above if necessary
final_results_with_fdr = final_results_with_fdr.drop(columns=[0], axis=1)

# Save results to CSV
final_results.columns = ["X", "Y", "Zone", "field_mean", "pop_mean", "diff_mean", "SE_mean", "t_mean", "p_value_mean",
                         "Adj_Sig_mean", "field_slope", "pop_slope", "diff_slope", "SE_slope", "t_slope",
                         "p_value_slope", "Adj_Sig_slope"]
final_results.to_csv("gdrive/MyDrive/output/comparison_analysis_zones_rpms_python_ver.csv", index=False)

print(f"Processing time: {time.time() - start_time} seconds")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


2024-09-30 18:26:45,502 - INFO - Zones to be processed: [1168 1171 1173 1170 1159 1148 1152 1153 1166 1169 1160 1161 1176 1174
 1167 1162 1165 1163 1164 1155 1156 1158 1146 1149 1144 1172 1154]
2024-09-30 18:26:45,505 - INFO - Processing zone: 1168
2024-09-30 18:26:45,555 - INFO - Processing zone: 1171
2024-09-30 18:26:45,639 - INFO - Processing zone: 1173


             X          Y  Zone  value_band_1  value_band_2  value_band_3  \
0  -106.222369  38.307806  1168         880.0          15.0        1980.0   
1  -106.221830  38.307806  1171         880.0           0.0        1882.0   
2  -106.221291  38.307806  1171         874.0           0.0        1950.0   
3  -106.220752  38.307806  1173         874.0           0.0        1984.0   
4  -106.220213  38.307806  1173         680.0           0.0        2020.0   
..         ...        ...   ...           ...           ...           ...   
95 -105.907060  38.307267  1164        1608.0        1958.0        1857.0   
96 -105.906521  38.307267  1159        1284.0        1703.0        1445.0   
97 -105.905443  38.307267  1161        2213.0        2512.0        3126.0   
98 -105.903826  38.307267  1164        1315.0        1669.0        1818.0   
99 -105.903287  38.307267  1165        2606.0        3589.0        4440.0   

    value_band_4  value_band_5  value_band_6  value_band_7  ...  \
0       

2024-09-30 18:26:45,818 - INFO - Processing zone: 1170


y [ 680.         1497.72222222 2020.         1424.         1954.
 1317.         1345.         2076.         1733.         1549.
 1488.         1638.         1171.         1388.         2073.
 2588.         2571.         2277.          858.          957.
 1784.         1066.         1397.         1464.         1697.
 1089.         1186.         1567.         1353.         1325.
 1957.         1441.          956.         1168.         1507.
  797.         1152.         1627.          978.        ]
39
            X          Y  Zone   mean_field  mean_population  mean_difference  \
0 -106.220213  38.307806  1173  1490.146724      1149.893208       340.253516   

    se_mean  t_stat_mean  p_value_mean  adj_p_value_mean  slope_field  \
0  11.67169    29.152034           0.0               NaN    -8.130185   

   slope_population  slope_difference  se_slope  t_stat_slope  p_value_slope  \
0          4.385312        -12.515496  3.404877     -3.675756       0.000748   

   adj_p_value_slope  
0 

...

2024-09-30 18:26:47,438 - INFO - Processing zone: 1155
2024-09-30 18:26:47,469 - INFO - Processing zone: 1156
2024-09-30 18:26:47,499 - INFO - Processing zone: 1158
2024-09-30 18:26:47,549 - INFO - Processing zone: 1146


           X          Y  Zone  mean_field  mean_population  mean_difference  \
0 -106.21644  38.307267  1164  801.560541      1446.823216      -645.262675   

    se_mean  t_stat_mean  p_value_mean  adj_p_value_mean  slope_field  \
0  7.406616   -87.119765           0.0               NaN    -2.957564   

   slope_population  slope_difference  se_slope  t_stat_slope  p_value_slope  \
0         -2.428514          -0.52905  2.161985     -0.244705       0.808037   

   adj_p_value_slope  
0                NaN  
y [ 837.  317.  933.  970. 1123.  769.  764.  898. 1033.  831. 1037. 1169.
  652. 1136.  899. 1295.  757. 1065.  525.  850.  809.  755.  721. 1087.
  944.  904.  674.  675.  819.  742. 1524. 1110.  947.  666. 1099.  826.
 1483.  834. 1164.]
39
            X          Y  Zone  mean_field  mean_population  mean_difference  \
0 -106.047197  38.307267  1164  913.923077      1446.823216      -532.900139   

    se_mean  t_stat_mean  p_value_mean  adj_p_value_mean  slope_field  \
0  9.3341

2024-09-30 18:26:47,618 - INFO - Processing zone: 1149
2024-09-30 18:26:47,665 - INFO - Processing zone: 1144
2024-09-30 18:26:47,695 - INFO - Processing zone: 1172
2024-09-30 18:26:47,753 - INFO - Processing zone: 1154


            X          Y  Zone  mean_field  mean_population  mean_difference  \
0 -106.230993  38.307267  1146  507.527903       727.442634      -219.914731   

    se_mean  t_stat_mean  p_value_mean  adj_p_value_mean  slope_field  \
0  6.637544   -33.131944           0.0               NaN     3.840475   

   slope_population  slope_difference  se_slope  t_stat_slope  p_value_slope  \
0         -0.271916          4.112392  1.928976      2.131904       0.039719   

   adj_p_value_slope  
0                NaN  
y [ 689.   883.6 1196.   678.  1074.   813.   883.6  639.   695.   898.
  819.  1004.   707.   789.  1682.  1034.   947.   932.   719.   734.
  723.   726.   846.   990.   816.   801.  1043.   770.   870.   805.
 1207.  1057.   751.   852.  1028.   677.  1026.  1021.   753. ]
39
            X          Y  Zone  mean_field  mean_population  mean_difference  \
0 -106.229376  38.307267  1146  886.620513       727.442634       159.177878   

    se_mean  t_stat_mean  p_value_mean  adj_

IndexError: index 2 is out of bounds for axis 0 with size 2